In [2]:
from collections import Counter

import pandas as pd
import statbotics

sb = statbotics.Statbotics()

In [3]:
all_matches = []
events_1 = sb.get_events(year=2022, week=1, fields=["key"])
events_2 = sb.get_events(year=2022, week=2, fields=["key"])
events_3 = sb.get_events(year=2022, week=3, fields=["key"])
for week, events in enumerate([[], events_1, events_2, events_3]):
    for event in events:
        try:
            matches = sb.get_matches(event=event["key"])
            for match in matches:
                match["week"] = week
            all_matches.extend(matches)
        except Exception:
            pass
matches_df = pd.DataFrame(all_matches)
matches_df.head()

,key,year,event,comp_level,set_number,match_number,playoff,time,status,red,...,blue_1,red_2,blue_2,red_endgame,blue_endgame,red_fouls,blue_fouls,red_no_fouls,blue_no_fouls,week
0,2022bcvi_qm1,2022,2022bcvi,qm,1,1,False,1646426638,Completed,"7332,4421,7787",...,3.0,0.0,0.0,15.0,6.0,0.0,4.0,35.0,11.0,1
1,2022bcvi_qm2,2022,2022bcvi,qm,1,2,False,1646427312,Completed,"7287,7190,8338",...,18.0,0.0,0.0,0.0,4.0,0.0,0.0,7.0,26.0,1
2,2022bcvi_qm3,2022,2022bcvi,qm,1,3,False,1646428102,Completed,"216,7167,7173",...,13.0,0.0,0.0,0.0,6.0,0.0,0.0,2.0,25.0,1
3,2022bcvi_qm4,2022,2022bcvi,qm,1,4,False,1646428873,Completed,"6485,244,8082",...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,28.0,6.0,1
4,2022bcvi_qm5,2022,2022bcvi,qm,1,5,False,1646429430,Completed,"288,7167,216",...,14.0,0.0,0.0,6.0,0.0,0.0,0.0,23.0,20.0,1


In [15]:
out = []
for _, m in matches_df[(matches_df.red_no_fouls >= 100) | (matches_df.blue_no_fouls >= 100)].iterrows():
    for alliance in ["red", "blue"]:
        if m[alliance + "_no_fouls"] < 100:
            continue
            
        out.append({
            "teams": [int(x) for x in m[alliance].split(",")],
            "event": m.event,
            "week": m.week,
            "comp_level": m.comp_level,
            "set_number": m.set_number,
            "match_number": m.match_number,
            "alliance": alliance,
            "auto": int(m[alliance + "_auto"]),
            "teleop": int(m[alliance + "_teleop"]),
            "endgame": int(m[alliance + "_endgame"]),
            "score": int(m[alliance + "_no_fouls"]),
            "cargo_rp": bool(m[alliance + "_rp_1"]),
            "hanger_rp": bool(m[alliance + "_rp_2"]),
        })
        
matches_100_df = pd.DataFrame(out)
display(matches_100_df.sort_values(by=["score"], ascending=False).head(20))

teams = []
for t in matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] > 6], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,cargo_rp,hanger_rp
95,"[6740, 1577, 1690]",2022isde3,3,qm,1,40,red,36,102,25,163,True,True
86,"[2212, 1690, 3075]",2022isde3,3,qm,1,18,red,32,103,21,156,True,True
104,"[1577, 1690, 1576]",2022isde3,3,f,1,2,red,34,99,14,147,False,False
103,"[1577, 1690, 1576]",2022isde3,3,f,1,1,red,30,98,19,147,False,False
13,"[1577, 1690, 6104]",2022isde1,1,qf,1,2,red,30,101,15,146,False,False
88,"[7039, 5614, 1690]",2022isde3,3,qm,1,21,red,38,72,31,141,True,True
92,"[2231, 4586, 1690]",2022isde3,3,qm,1,30,blue,30,80,30,140,True,True
5,"[4414, 1678, 589]",2022caph,1,qf,1,2,red,34,86,16,136,False,False
160,"[2930, 2910, 8532]",2022wayak,3,f,1,1,red,34,70,30,134,False,False
98,"[1690, 2630, 3083]",2022isde3,3,qm,1,54,blue,34,78,21,133,True,True


[(1690, 20),
 (1577, 16),
 (4414, 10),
 (2910, 10),
 (27, 8),
 (177, 8),
 (176, 8),
 (1678, 7),
 (5813, 7)]

In [10]:
qm_matches_100_df = matches_100_df[matches_100_df.comp_level == "qm"]
display(qm_matches_100_df.sort_values(by=["score"], ascending=False).head(20))

teams = []
for t in qm_matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] > 2], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,cargo_rp,hanger_rp
95,"[6740, 1577, 1690]",2022isde3,3,qm,1,40,red,36,102,25,163,True,True
86,"[2212, 1690, 3075]",2022isde3,3,qm,1,18,red,32,103,21,156,True,True
88,"[7039, 5614, 1690]",2022isde3,3,qm,1,21,red,38,72,31,141,True,True
92,"[2231, 4586, 1690]",2022isde3,3,qm,1,30,blue,30,80,30,140,True,True
98,"[1690, 2630, 3083]",2022isde3,3,qm,1,54,blue,34,78,21,133,True,True
96,"[7845, 1690, 6738]",2022isde3,3,qm,1,44,blue,34,69,30,133,True,True
0,"[1678, 973, 5817]",2022caph,1,qm,1,2,red,20,81,27,128,True,True
80,"[111, 5822, 4143]",2022ilpe,3,qm,1,68,blue,32,59,36,127,True,True
75,"[111, 1756, 4655]",2022ilpe,3,qm,1,19,red,30,68,27,125,True,True
126,"[2960, 3357, 2075]",2022mimus,3,qm,1,49,red,34,54,36,124,True,True


[(1690, 10),
 (1577, 6),
 (2363, 4),
 (2910, 3),
 (2481, 3),
 (1756, 3),
 (111, 3),
 (6738, 3),
 (1987, 3),
 (5813, 3)]

In [12]:
elim_matches_100_df = matches_100_df[matches_100_df.comp_level != "qm"]
display(elim_matches_100_df.sort_values(by=["score"], ascending=False).head(20))

teams = []
for t in elim_matches_100_df.teams:
    teams.extend(t)
sorted([x for x in dict(Counter(teams)).items() if x[1] > 4], key=lambda x:-x[1])

,teams,event,week,comp_level,set_number,match_number,alliance,auto,teleop,endgame,score,cargo_rp,hanger_rp
104,"[1577, 1690, 1576]",2022isde3,3,f,1,2,red,34,99,14,147,False,False
103,"[1577, 1690, 1576]",2022isde3,3,f,1,1,red,30,98,19,147,False,False
13,"[1577, 1690, 6104]",2022isde1,1,qf,1,2,red,30,101,15,146,False,False
5,"[4414, 1678, 589]",2022caph,1,qf,1,2,red,34,86,16,136,False,False
160,"[2930, 2910, 8532]",2022wayak,3,f,1,1,red,34,70,30,134,False,False
4,"[4414, 1678, 589]",2022caph,1,qf,1,1,red,34,68,30,132,False,False
12,"[1577, 1690, 6104]",2022isde1,1,qf,1,1,red,38,80,10,128,False,False
49,"[359, 4414, 5089]",2022cave,2,sf,1,1,red,30,74,24,128,False,False
101,"[1577, 1690, 1576]",2022isde3,3,sf,1,1,red,30,81,15,126,False,False
100,"[1577, 1690, 1576]",2022isde3,3,qf,1,2,red,34,58,34,126,False,False


[(1577, 10),
 (1690, 10),
 (4414, 9),
 (177, 8),
 (176, 8),
 (2910, 7),
 (27, 6),
 (1576, 6),
 (7127, 6),
 (1678, 5),
 (589, 5)]